In [1]:
from pyvis import network as net
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import rcParams, cycler
from matplotlib.lines import Line2D
import matplotlib.cm as cmx
import collections

In [2]:
all_df = pd.read_csv("Phase5_Data.csv")

#Categories needed for each person
#Submitter, medium, mood after conversation, extroversion scale, category of person
interaction_types = ["Text"]

temp_df = all_df[['Submitter', 'Medium of Conversation', 'Mood after the conversation', 
                  'Extroversion Scale', 'General Category of Person']]

jessie_df = temp_df.query('Submitter == "Jessie"')
shaina_df = temp_df.query('Submitter == "SR"')
erin_df = temp_df.query('Submitter == "Erin"')
luke_df = temp_df.query('Submitter == "Luke"')

#print(len(jessie_df.query('`General Category of Person` == "Friend"')))
#jessie_df = jessie_df.head(5)
#print(len(jessie_df))


categories = ['School','Family','Friend','Stranger','Other','Work']

def categoryPersonPercents(submitter_df):
    
    res = {}
    submitterTotal = len(submitter_df)
    for category in categories:
        temp_df = submitter_df[submitter_df["General Category of Person"] == category]
        categoryTotal = len(temp_df)
        moodAverage = temp_df["Mood after the conversation"].mean()        
        percentage = (categoryTotal/submitterTotal)*100
        if (percentage != 0):
            res[category] = moodAverage
    
    
    #res = sorted(res.items(), key=lambda x: x[1])    
    return res



jessie_categories = categoryPersonPercents(jessie_df)
shaina_categories = categoryPersonPercents(shaina_df)
erin_categories = categoryPersonPercents(erin_df)
luke_categories = categoryPersonPercents(luke_df)

    


In [3]:
categories_color={
    'Friend': '#4BC6C7', #Blue
    'Stranger': '#a39c9b', #Gray
    'Work': '#FFC300', #Yellow
    'School': '#EE3110',#Red
    'Family': '#80CD0A', #Green
    'Other': '#7511D8' #Purple   
}

submitter_shape = {
    'Jessie': 'square', #Blue
    'Shaina': 'diamond', #Yellow
    'Erin': 'triangle',#Red
    'Luke': 'dot', #Green  
}
submitter_color={
    'Jessie': '#4BC6C7', #Blue
    'Shaina': '#FFC300', #Yellow
    'Erin': '#EE3110',#Red
    'Luke': '#80CD0A', #Green  
}

category_shapes={
    'Friend': 'dot', #Blue
    'Stranger': 'diamond', #Gray
    'Work': 'triangle', #Yellow
    'School': 'square',#Red
    'Family': 'star', #Green
    'Other': 'triangleDown' #Purple   
}

extraversion = {
    "Jessie": 4,
    "Erin": 5,
    "Shaina": 3,
    "Luke": 7
}

node_sizes = {
    1: 10,
    2: 20,
    3: 30,
    4: 40,
    5: 50,
    6: 60,
    7: 70,
    8: 80,
    9: 90,
    10: 100
}

# Find submitter's percentage for each type of person interaction
percentages = []
categories = ['School','Family','Friend','Stranger','Other','Work']




def createSettings():
    return net.Network(notebook=True, height='800px', width='950px')

"""
def createGraph(submitter, submitter_df, shape):
    #Add central name node
    g = nx.Graph()
    g.add_node(-1, label=submitter, size=8, title="central node", color='black',group="central")
    #g.repulsion(node_distance=50, spring_length=200)
    msg = "placeholder"
    for index, row, in submitter_df.iterrows():
        personCategory = row['General Category of Person']
        msg = "<p><b>Category</b>: " + personCategory +"</p>"
        g.add_node(index, label=row['Medium of Conversation'], 
                   size=node_sizes[int(row['Mood after the conversation'])],
                   title=msg,
                   group="datapoint",
                   value=node_sizes[int(row['Mood after the conversation'])]*10,
                   color=categories[personCategory],
                   mass=int(row['Mood after the conversation']),
                   shape=shape)        
        g.add_edge(-1, index)
    return g
"""


def createGraph(submitter, category_res, shape, g = None):
    #Add central name node
    if (g == None):
        g = nx.Graph()
   
    print("NOT NONE")
    g.add_node(-1, label=submitter, color="black", size=20, title="central node", group="central")
    #g.repulsion(node_distance=50, spring_length=200)
    msg = "placeholder"
    index = 0
    for item in category_res.items():
        labelMsg = f"{item[0]} ({round(item[1], 2)})"
        msg = f"<p><b>Category: </b>{item[0]} <br><b>Average mood: </b>{round(item[1],2)}<br><b>Extroversion: </b>{extraversion[submitter]}</p>"
        print(item)
        g.add_node(index, 
                   label=labelMsg,                   
                   #Size = total propoprtion
                   size=item[1]*10,
                   title=msg,
                   group="datapoint",
                   #value=value[1]*10,
                   color=categories_color[item[0]],
                   mass=item[1],
                   shape=shape
                )    
        g.add_edge(-1, index, length=item[1]*item[1]*10)
        index+=1
        
    return g

In [4]:
#Legend

i = 0
j = 1
gLegend = nx.Graph()
topRow = []
bottomRow = []

for key in node_sizes.keys():
    print(i,j)
    msg = f"Mood: {key}"
    gLegend.add_node(i, label='Central node', size=20, group=key, mass=key, shape="dot", color='black')
    gLegend.add_node(j, label=msg, size=key*10, group=key, mass=key, shape = "square", color='black')
    gLegend.add_edge(i, j, length=key*key*10)
    topRow.append(i)
    bottomRow.append(j)
    
    i+=2
    j+=2
    
print(topRow)
for i in range(0, len(topRow)-1):
    gLegend.add_edge(topRow[i], topRow[i+1])
    gLegend.add_edge(bottomRow[i], bottomRow[i+1])
    


legend = createSettings()
legend.from_nx(gLegend)
legend.show("legend.html")




0 1
2 3
4 5
6 7
8 9
10 11
12 13
14 15
16 17
18 19
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In [5]:
g1 = createSettings()
nxg = createGraph("Jessie", jessie_categories, "square")
g1.from_nx(nxg)
g1.show('jessie.html')

NOT NONE
('School', 6.375)
('Family', 6.333333333333333)
('Friend', 6.954545454545454)
('Work', 6.166666666666667)


In [6]:
g2 = createSettings()
nxg = createGraph("Shaina", shaina_categories, "diamond")
g2.from_nx(nxg)
g2.show('shaina.html')

NOT NONE
('School', 7.333333333333333)
('Family', 7.173913043478261)
('Friend', 7.1)
('Stranger', 6.5)


In [7]:
g3 = createSettings()
nxg = createGraph("Erin", erin_categories, "triangle")
g3.from_nx(nxg)
g3.show('erin.html')

NOT NONE
('School', 7.0)
('Family', 6.333333333333333)
('Friend', 7.785714285714286)
('Stranger', 6.375)
('Work', 5.777777777777778)


In [8]:
g4 = createSettings()
nxg = createGraph("Luke", luke_categories, "dot")
g4.from_nx(nxg)
g4.show('luke.html')

NOT NONE
('School', 6.166666666666667)
('Family', 8.75)
('Friend', 8.571428571428571)
('Stranger', 6.2)
('Other', 6.846153846153846)
('Work', 7.625)


In [9]:
g5 = createSettings()
#g5.show_buttons(filter_=['nodes'])
submitters = [jessie_categories, shaina_categories, erin_categories, luke_categories]
g = nx.Graph()
g.add_node(-1, label="ALL", color="black", size=20, title="Central node", group="central")

index = 0
outerIndex = 0
shapes = ["square", "diamond", "triangle", "dot"]
msg = "trst"
categories = ['School','Family','Friend','Stranger','Other','Work']

submitter_color2={
    'jessie_categories': '#4BC6C7', #Blue
    'shaina_categories': '#FFC300', #Yellow
    'erin_categories': '#EE3110',#Red
    'luke_categories': '#80CD0A', #Green  
}


def addNodes(g, item, submitter, df, index, prevIndex, start=None):

    if item in df.keys():
        #print(categories_color[item])
        labelMsg = f"{item} ({round(df[item], 2)})"
        msg = f"<p><b>Category: </b>{item} <br><b>Average mood: </b>{round(df[item],2)}<br><b>Extroversion: </b>{extraversion[submitter]}<br><b>Submitter:</b> {submitter}</p>"
        g.add_node(index, 
                       label=labelMsg,                   
                       #Size = total propoprtion
                       size=df[item]*5,
                       title=msg,
                       group=submitter,
                       #value=value[1]*10,
                       color=categories_color[item],
                       mass=df[item],
                       width=1,
                       shape=submitter_shape[submitter]
                    )
        if (prevIndex != index):
  
            g.add_edge(prevIndex, index)
        if (start != None):
            g.add_edge(index, start)
       
        prevIndex = index
        index+=1
    
    return (index, prevIndex)


#Link all the same category nodes
for item in categories:

    prevIndex=-1
    
    start = index
    indexes = addNodes(g, item, 'Jessie', jessie_categories, index, prevIndex)
    indexes = addNodes(g, item, 'Shaina', shaina_categories, indexes[0], indexes[1])
    indexes = addNodes(g, item, 'Luke', luke_categories, indexes[0], indexes[1])
    indexes = addNodes(g, item, 'Erin', erin_categories, indexes[0], indexes[1], start)
    
    index = indexes[0]+1
    

            
"""
for categories in submitters:    
    for item in categories:
        print(item)
        g.add_node(index, 
                   label=item[0],                   
                   #Size = total propoprtion
                   size=item[1]*10,
                   #title=msg,
                   group=outerIndex,
                   value=item[1]*10,
                   color=categories_color[item[0]],
                   mass=item[1],
                   shape=shapes[outerIndex]
                )    
        g.add_edge(-1, index, value = item[1]*10)
        index+=1
    outerIndex+=1
"""

g5.from_nx(g)
g5.show('all.html')



Current mappings:
- Color: category
- Length of edge: mood
- Size of node: mood

Revised mapping:

Size of node: # of interactions for that category (percentage of interactions)

- Extraversion/Submitter
    - Shape
- Mood:
    - Length of the edge
- Different categories of people (6 nodes)
    - Single node
    - Color + label

Tooltip (extra variables)

